<a href="https://colab.research.google.com/github/Vijaycriss/MNSIT_Tensorflow_DNN/blob/main/Audiobooks_customer_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf

from sklearn import preprocessing

In [6]:
raw_data = np.loadtxt('/content/Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_data[:,1:-1]
targets_all = raw_data[:,-1]

In [7]:
shuffled_indicies = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indicies)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indicies]
targets_all = targets_all[shuffled_indicies]

##Balance the data

In [8]:
#count how many targets are 1
num_of_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

indices_to_remove = []

#count the number of 0
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
      zero_targets_counter += 1
      if zero_targets_counter > num_of_targets:
        indices_to_remove.append(i)

unscaled_inputs_equal = np.delete(unscaled_inputs_all, indices_to_remove, axis =0)
targets_equal = np.delete(targets_all, indices_to_remove, axis = 0)


##Standarize the input

In [9]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal)

##Shuffle data

In [11]:
shuffled_indicies = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indicies)

shuffled_inputs = scaled_inputs[shuffled_indicies]
shuffled_targets = targets_equal[shuffled_indicies]

##Split train,val and test data

In [13]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)



1801.0 3579 0.5032131880413523
214.0 447 0.47874720357941836
222.0 448 0.4955357142857143


##Save in *.npz

In [14]:
np.savez('Train_data', inputs = train_inputs, targets = train_targets)
np.savez('Validation_data', inputs = validation_inputs, targets = validation_targets)
np.savez('test_data', inputs = test_inputs, targets = test_targets)

##Outline of model

Load Data

In [ ]:
npz = np.load('/content/Train_data.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('/content/Validation_data.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('/content/test_data.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

##Model

In [31]:
input_size = 10
output_size = 2
hidden_layers_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layers_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layers_size, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax')
])

##Opitmizer and loss func

In [32]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100

early_stop = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks = [early_stop],
          validation_data = (validation_inputs, validation_targets),
          verbose = 2)

Epoch 1/100
36/36 - 1s - loss: 0.6205 - accuracy: 0.6535 - val_loss: 0.5282 - val_accuracy: 0.7539 - 1s/epoch - 33ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4889 - accuracy: 0.7647 - val_loss: 0.4533 - val_accuracy: 0.7785 - 139ms/epoch - 4ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4368 - accuracy: 0.7790 - val_loss: 0.4199 - val_accuracy: 0.7785 - 159ms/epoch - 4ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4115 - accuracy: 0.7904 - val_loss: 0.4048 - val_accuracy: 0.7897 - 184ms/epoch - 5ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3962 - accuracy: 0.7977 - val_loss: 0.3953 - val_accuracy: 0.7763 - 165ms/epoch - 5ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3873 - accuracy: 0.7991 - val_loss: 0.3892 - val_accuracy: 0.7785 - 150ms/epoch - 4ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3815 - accuracy: 0.8013 - val_loss: 0.3849 - val_accuracy: 0.7830 - 146ms/epoch - 4ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3766 - accuracy: 0.8058 - val_loss: 0.3803 - val_accuracy: 0.7964 - 147ms/epoch - 4ms/step
Epoch 9/10

##Test the model

In [33]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('Test loss : {0:.2f}, Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

14/14 [==============================] - 0s 4ms/step - loss: 0.3565 - accuracy: 0.8170
Test loss : 0.36, Test accuracy: 81.70%
